# Agentic Self-Reflective RAG on Dell AI Factory with NVIDIA
### with Elasticsearch vector database
### Models served from K8s cluster

<img src="images/agentic-rag-pipeline.png" alt="Alternative text" />

## What is Agentic RAG?  

LLM agents extend the capabilities of traditional LLMs by blending their natural language comprehension capabilities with actionable functionalities. This is a significant advancement for AI, making it ideal for automation and intelligent decision-making across industries. Unlike traditional LLMs, which generate text based solely on their training data, LLM agents can connect to external systems such as APIs, databases, and applications to fetch live data, provide contextually relevant responses, and combine them in pipelines to enhance their utility in real-world applications.


This ability transforms LLMs from passive responders into dynamic actors capable of handling multi-step workflows and delivering actionable insights. In healthcare, for instance, LLM agents can securely synthesize information from patient records, clinical guidelines, and research databases to support timely, evidence-based decisions. These agents can assist in tasks such as patient diagnosis, treatment planning, and drug discovery, thereby enhancing the efficiency and accuracy of healthcare processes.


The power to process and act on information in real time while adhering to stringent compliance standards positions LLM agents as powerful tools for addressing complex, data-intensive challenges. The agents redefine what AI can accomplish, providing scalable, secure, and contextually relevant solutions to some of the most demanding problems in modern industries.


# NVIDIA NIMs

The `langchain-nvidia-ai-endpoints` package contains LangChain integrations building applications with models on 
NVIDIA NIM inference microservice. NIM supports models across domains like chat, embedding, and re-ranking models 
from the community as well as NVIDIA. These models are optimized by NVIDIA to deliver the best performance on NVIDIA 
accelerated infrastructure and deployed as a NIM, an easy-to-use, prebuilt containers that deploy anywhere using a single 
command on NVIDIA accelerated infrastructure.

NVIDIA hosted deployments of NIMs are available to test on the [NVIDIA API catalog](https://build.nvidia.com/). After testing, 
NIMs can be exported from NVIDIA’s API catalog using the NVIDIA AI Enterprise license and run on-premises or in the cloud, 
giving enterprises ownership and full control of their IP and AI application.

### About this notebook

- Single LLM role play in a multi-agent set of tasks
- Two data sources are used, RAG and a web search fall back, but more can be added to the query router.  Route A and B are available.  Route C is shown as an example.
- NVIDIA NIMS are installed on a K8s cluster and accessed via API calls
- Notebook does not need to be run on a GPU enabled machine, all GPU required services are provided by the K8s cluster.
- Features code that can assist with clickable source files
- Features a method to turn OFF the Agentic processes to show the different in results.

### Code credit and inspiration:
- https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_self_rag/#llms
- https://github.com/NVIDIA/workbench-example-agentic-rag
- David O'Dell
- Tiffany Fahmy

# Library installs

In [1]:
# %pip install -q langchain-nvidia-ai-endpoints==0.2.2
# %pip install -q langchain==0.2.16
# %pip install -q langchain-community==0.2.17                   
# %pip install -q langchain-core==0.2.40
# %pip install -q langchain-text-splitters==0.2.4
# %pip install -q langchain-openai==0.1.23
# %pip install -q pdfminer-six==20231228
# %pip install -q pillow-heif==0.18.0
# %pip install -q opencv-python==4.10.0.84 
# %pip install -q unstructured==0.15.9
# %pip install -q unstructured-pytesseract==0.3.12
# %pip install -q pi-heif==0.18.0
# %pip install -q unstructured-inference==0.7.36
# %pip install -q tesseract==0.1.3
# %pip install -q pytesseract==0.3.10
# %pip install -q langgraph==0.2.15
# %pip install -q gradio==4.27.0
# %pip install -q elasticsearch==8.15.1
# %pip install -q tiktoken==0.8.0
# %pip install -q langchain-elasticsearch==0.2.2

### Set debug and verbosity

In [2]:
# from langchain.globals import set_verbose, set_debug

# set_debug(True)
# set_verbose(True)

# Import Libraries

In [3]:
import nltk  
print(nltk.__version__)

3.9.1


In [4]:
### import loaders
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import CSVLoader
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_community.document_loaders import OnlinePDFLoader
from langchain_community.document_loaders.merge import MergedDataLoader

### for embedding
# from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

# from langchain_community.vectorstores import Chroma

### status bars and UI and other accessories
from tqdm import tqdm
import time

# Declare external services

Services that will be hosted outside this application, usually the LLM, the vectordb and anything else.

## Langsmith Tracing Setup

In [5]:
import os

### Consider adding these as env vars in AI Workbench to enable LangSmith tracing ###
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ["LANGCHAIN_PROJECT"] = "YOUR PROJECT NAME"
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "YOUR API KEY"


### Define Local LLM for initial testing

##### Model NIM, Embeddingn and Rerank will all have different ports.  In this case we used 30001, 30002, 30003. 

In [7]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, NVIDIARerank
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import VLLM
from langchain_openai import OpenAI

In [8]:
model_id = "meta/llama-3.1-8b-instruct"
api_url = "http://YOUR MODEL SERVER AND PORT/v1"


llm = ChatOpenAI(
    base_url=api_url,
    api_key="YOUR API KEY",
    model=model_id,
    temperature=0,
    max_tokens=None,
)

### Define embeddings options

In [9]:
embeddings = NVIDIAEmbeddings(
    base_url="http://YOUR MODEL SERVER AND PORT/v1", 
    model="nvidia/nv-embedqa-e5-v5",
    truncate="END"
)

### Define reranking options

In [10]:
reranker = NVIDIARerank(
    base_url="YOUR MODEL SERVER AND PORT/v1", 
    model="nvidia/nv-rerankqa-mistral-4b-v3",
    truncate="END"
)

## Define Elasticsearch vector db instance

using this as inspiration:  https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/generative-ai/chatbot.ipynb

In [11]:
from elasticsearch import Elasticsearch
from langchain_elasticsearch import ElasticsearchStore

In [12]:
### set certificate permissions to 644

In [13]:
CERTIFICATE = "/home/demo/es_http_ca.crt"
HOST = "https://localhost:9200"
USER = "elastic"
PASSWORD = "PASSWORD"

In [14]:
es_client = Elasticsearch(
    hosts=HOST,
    basic_auth=(USER, PASSWORD),
    verify_certs=True,
    ca_certs=CERTIFICATE,
    # verify_certs=False,
    # ca_certs=False,
    # ca_certs=True,    
    # connection_class=RequestsHttpConnection,
)

In [15]:
print(es_client.ping())
print(es_client)
print(es_client.info())

True
<Elasticsearch(['https://localhost:9200'])>
{'name': '0a7b0a930bd9', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Xq58NyVaSeq80WPUT2CTpg', 'version': {'number': '8.15.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'f97532e680b555c3a05e73a74c28afb666923018', 'build_date': '2024-10-09T22:08:00.328917561Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


# Vector db Content setup

### Load PDF data into loader object

##### We want the pdf files to be clickable so we set up a prefix appended to each file that points to the server they are residing at.

In [19]:
# Directory containing PDF files
pdf_directory = "docs/pdf-powerscale-mount"
url_prefix = "http://IP ADDRESS OF FILE SERVER/"

# Load PDF documents
pdf_dir_loader = PyPDFDirectoryLoader(pdf_directory)


### Load CSV data into loader object

In [20]:
patient_data_csv_loader = CSVLoader("docs/csv-powerscale-mount/healthcare.csv", encoding='windows-1252')

### view CSV head contents

In [21]:
import pandas as pd

In [22]:
df = pd.read_csv('docs/csv-powerscale-mount/healthcare.csv')
print(df.head(5))

  Patient Name Date of Birth  Age Reason for Admission       Procedure  \
0    Patient_3      8/6/2004   19                COVID  antiviral meds   
1   Patient_39     3/19/2004   19                COVID  oxygen therapy   
2   Patient_33     12/8/2002   21                COVID  oxygen therapy   
3   Patient_40      7/6/2001   23                  Flu  antiviral meds   
4    Patient_1    10/28/1999   24            Pneumonia     antibiotics   

       Room Date of Discharge  Length of Stay   Charges  Balance Remaining  \
0  Room_237        12/27/2023               0    237.34              37.34   
1  Room_440         1/12/2024              22  29980.84            3018.84   
2  Room_298        12/31/2023               6   4028.77            2681.26   
3  Room_360        11/30/2023               0    273.69               0.00   
4  Room_239          1/5/2024               1    490.50             341.28   

   Unnamed: 10  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4

In [23]:
num_rows = df.shape[0]
print(f"Total number of rows: {num_rows}")

Total number of rows: 50


## merge pdf and csv

In [24]:
# Merge the PDF and CSV loaders into a single dataset
merged_loader = MergedDataLoader(loaders=[pdf_dir_loader, patient_data_csv_loader])

# Load all the merged documents
merged_documents = merged_loader.load()

/home/demo/miniconda3/envs/rag/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [25]:
# len(merged_documents)

In [26]:
### CSV file rows are broken down and made into one document per row
### 230 PDf file documents, + 50 rows of CSV file = 280 documents

In [27]:
# merged_documents[0]

## Transform source format to include URL for pdf chunks in documents 
This assumes an nginx instance running and pointing to a mounted pdf directory

In [28]:
# Prepend URL prefix to the source in metadata

pdf_count = 0
csv_count = 0

for doc in merged_documents:
    if 'source' in doc.metadata:
        # Remove the directory part from the source path
        file_name = doc.metadata['source'].replace(pdf_directory + "/docs", "")
        doc.metadata['source'] = url_prefix + file_name

        # Count the number of PDF and CSV documents
        if file_name.lower().endswith('.pdf'):
            pdf_count += 1
        elif file_name.lower().endswith('.csv'):
            csv_count += 1

# Print the total number of PDF and CSV documents
print(f"Total PDF documents: {pdf_count}")
print(f"Total CSV rows: {csv_count}")


# Print the updated sources to verify
for doc in merged_documents[:5]:  # Print first 5 for verification
    print(doc.metadata['source'])

Total PDF documents: 230
Total CSV rows: 50
http://172.16.6.3/docs/pdf-powerscale-mount/mental_health_first_aid.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/mental_health_first_aid.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_prevention_update.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_prevention_update.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_prevention_update.pdf


### Chunk and split documents

Each document will be chunked and split along the chunk_size parameter.  The overlap parameter will ADD to the amount of characters, so 512 plus 256 overlap will equal a split size of around 800.  An overlap of zero will equal a split size of only the chunk value.

In [29]:
# text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=512, chunk_overlap=64)
doc_splits = text_splitter.split_documents(merged_documents)

## Prepare Elasticsearch Index

In [30]:
INDEX_NAME = "merged_chunked_index"

### Delete and rebuild ES index

NOTE:  If you don't delete the index and start embedding duplicates into an existing index, you will get extremely bad performance and errors due to mulitiple documents with the same content.  

In [31]:
# Check if the index exists and delete it if it does
if es_client.indices.exists(index=INDEX_NAME):
    es_client.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' deleted successfully.")
else:
    print(f"Index '{INDEX_NAME}' does not exist, will be created in the next step.")

Index 'merged_chunked_index' deleted successfully.


### Initial embed documents into vector store

In [32]:
%%time

vectorstore = ElasticsearchStore.from_documents(
    doc_splits,
    embeddings,
    index_name=INDEX_NAME,
    es_connection=es_client,
)

print('\n' + 'Time to complete:')


Time to complete:
CPU times: user 376 ms, sys: 31.8 ms, total: 408 ms
Wall time: 3.1 s


### Verify document structure in Elasticsearch

In [33]:
# # Function to check the structure of documents in the index
# def check_document_structure(index_name, es_client, num_docs=2):
#     # Search for documents in the index
#     response = es_client.search(
#         index=index_name,
#         body={
#             "query": {
#                 "match_all": {}
#             },
#             "size": num_docs
#         }
#     )

#     # Check if documents are found
#     if response['hits']['total']['value'] > 0:
#         print(f"Found {response['hits']['total']['value']} documents in the index '{index_name}'.")
#         for doc in response['hits']['hits']:
#             print(f"Document ID: {doc['_id']}")
#             print(f"Document structure: {doc['_source']}")
#             print("-" * 80)
#     else:
#         print(f"No documents found in the index '{index_name}'.")

# # Check the structure of documents
# check_document_structure(INDEX_NAME, es_client)

### Create direct vectorstore retriever

In [34]:
retriever = vectorstore.as_retriever()

In [35]:
import json

def get_unique_files(merged_documents):
    file_list = []
    
    for doc in merged_documents:
        source = doc.metadata.get('source', None)
        if source:
            file_list.append(source)
    
    # Use a set to get unique file URLs
    unique_list = list(set(file_list))
    
    # Sort the unique list by file extension
    sorted_unique_list = sorted(unique_list, key=lambda x: x.split('.')[-1])
    
    print("\nList of unique files in merged loader, sorted by file type:\n")
    for unique_file in sorted_unique_list:
        print(unique_file)
    
    pretty_files = json.dumps(sorted_unique_list, indent=4, default=str)
    
    return pretty_files

# Example usage
unique_files_sorted = get_unique_files(merged_documents)



List of unique files in merged loader, sorted by file type:

http://172.16.6.3/docs/csv-powerscale-mount/healthcare.csv
http://172.16.6.3/docs/pdf-powerscale-mount/mental_health_first_aid.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_screening.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/investigating-the-efficacy-of-osimertinib-and-crizotinib-in-phase-3-clinical-trials-on-anti-cancer.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/covid_variants.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/understanding-pharmacology-covid-mrna.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_cells.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/ipilimumab-for-advanced-melanoma-a-pharmacologic-perspective.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/population_based_approach_to_mental_health.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_prevention_update.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/covid_update.pdf
http://172.16.6.3/docs/pdf-powerscal

# Setup and Test Agent pipeline elements

### Generate RAG Response

In [36]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant in a health care clinic. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Please keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
question = "Tell me about mental health from a population perspective."
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})


In [37]:
# print(generation)

### RAG only function for Basic RAG toggle

Grab the response and source doc name and a snippet of content

In [38]:
def get_rag_response(question):
    docs = retriever.invoke(question)
    generation = rag_chain.invoke({"context": docs, "question": question})
    

    basic_rag_formatted_output_source_docs = []

    base_url = 'http://172.16.6.3'

    thumbnail_path = 'images/thumbnails/folder-icon.jpg'
    
    for doc in docs:
        source = doc.metadata['source']  # Assuming 'metadata' is a dictionary with a 'source' key
        page_content_snippet = doc.page_content[:200]  # Get the first x number of characters of the snippet
        
        # Append the formatted HTML to the list
        basic_rag_formatted_output_source_docs.append(f'''
        <base href="{base_url}">

        <table>
            <tr>
                <td>
                <!-- <img src="{thumbnail_path}" alt="Thumbnail" style="width:100px;height:auto;"> -->
                
                <img src="{thumbnail_path}" alt="Thumbnail" style="width:auto;height:auto;">

                </td>
                <td>
                    <a href="{source}" target="_blank" class="custom-link">{source}</a><br>
                    Snippet: {page_content_snippet}<br><br>
                </td>
            </tr>
        </table>
        ''')

    # Combine the generation and formatted output into a single output
    return generation, basic_rag_formatted_output_source_docs


### Question Router


In [39]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
    user question to a vectorstore or web search. Use the vectorstore exclusively for questions related to patient data, skin cancer, covid, or mental health. 
    You do not need to be stringent with keywords in the question related to these topics. If **any document** is found to be relevant in the vectorstore, stop immediately and generate the answer using that data. 
    **Do not perform a web search** if even one relevant document is found, regardless of the overall assessment of other documents.
    If **no relevant data** is found at all in the vectorstore, or if the question is unrelated to these topics, use web_search.
    
    Provide the answer in JSON format with a single key called 'datasource' and a single answer either 'vectorstore' or 'websearch' as the value.
    Please do not include a preamble or explanation. Your response should be formatted as follows: \'{{"datasource": "value"}}\'.

    Example 1: A question that is not related to patient data, skin cancer, covid, or mental health should return with a response to use the web_search like this: \'{{"datasource": "websearch"}}\'.

    Example 2: A question that is related to patient data, skin cancer, covid, or mental health and any relevant data is found in the vectorstore should return a response like this: \'{{"datasource": "vectorstore"}}\'.

    Question to route: {question}
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)

question_router = prompt | llm | JsonOutputParser()
question = "Tell me about mental health from a population perspective."
docs = retriever.invoke(question)

doc_txt = docs[1].page_content
print(question_router.invoke({"question": question}))

{'datasource': 'vectorstore'}


In [40]:
# get_rag_response("Tell me about mental health from a population perspective")

### Relevance / Retrieval Grader

Checks index of vectorstore to see if there are relavent docs

In [41]:
from langchain.prompts import PromptTemplate
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import JsonOutputParser

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "Tell me about mental health from a population perspective."
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}


### Hallucination Grader

Checks to see if the generation is grounded in truth using the source documents as a reference.  
If the generation is grounded in truth, then the hallucination grader responds positively with Yes.

If the generation is NOT grounded in truth and has no relavence with the source documents, the grader responds negatively with No.

In [42]:
prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score in JSON format with a 
    single key 'score' and no preamble or explanation, like this \'{{\'"score": "yes"\'{{\' or \'{{\'"score": "no"\'{{\'. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'yes'}

### Answer Grader

Is the answer provided "useful" to the question.

In [43]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score in JSON format with a 
    single key 'score' and no preamble or explanation, like this \'{{\'"score": "yes"\'{{\' or \'{{\'"score": "no"\'{{\'. 
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})

{'score': 'yes'}

### Web Search

uses the python library for Tavily open search.  Create an account and API here:
https://blog.tavily.com/getting-started-with-the-tavily-search-api/

In [44]:
os.environ["TAVILY_API_KEY"] = "YOUR API KEY"

In [45]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

# Langgraph Node Functions Setup

In [46]:
from typing_extensions import TypedDict
from typing import List
from langchain.schema import Document

################################ State ##############################


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add internet search
        documents: list of documents
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]


In [47]:
################################ Nodes ##############################


def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    print(question)

    ## source here refers to which datasource to route to, RAG or web or other
    # Status message
    global router_status, router_choice
    
    target_source = question_router.invoke({"question": question})
    print(target_source)
    
    # Check if source is a dictionary
    if isinstance(target_source, dict):
        if "datasource" in target_source:
            print(target_source["datasource"])
            router_choice = target_source["datasource"]
            if target_source["datasource"] == "websearch":
                print("---DECISION: ROUTE QUESTION TO WEB SEARCH---")
                router_status = "success"
                return "websearch"
            elif target_source["datasource"] == "vectorstore":
                print("---DECISION: ROUTE QUESTION TO RAG---")
                router_status = "success"
                return "vectorstore"
        else:
            print("Error: 'datasource' key not found in source")
    else:
        print("Error: source is not a dictionary")

    return None

In [48]:
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE USING NVIDIA EMBEDDINGS NIM---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    
    # Status message
    global retrieve_status
    retrieve_status = "success"
    
    return {"documents": documents, "question": question}

In [49]:
def rerank(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---NVIDIA RERANK NIM PROCESS---")
    question = state["question"]
    documents = state["documents"]

    # Reranking
    documents = reranker.compress_documents(query=question, documents=documents)

    # Status message
    global rerank_status
    rerank_status = "success"
    
    return {"documents": documents, "question": question}

In [50]:
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.
    If no document is relevant, we will set a flag to run web search.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    
    global filtered_docs  # Declare the global variable to place docs in to be accessed in other functions

    # Score each doc
    filtered_docs = []

    web_search = "Yes"  # Default to Yes in case there is no relevant doc

    ## take the page content value of documents retrieved and grade it against the question,
    ## this means the filtered_docs array will only contain page content values, not file names
    
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score["score"]
        print(grade)
        
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
            # Since we found at least one relevant document, set web_search to "No"
            web_search = "No"
            
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # Do not include the document in filtered_docs
            # will default to web search = yes
            continue

    # Status message
    global relevance_status
    relevance_status = "success"
    
    return {"documents": filtered_docs, "question": question, "web_search": web_search}



In [51]:
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    global web_fallback_status
    
    if web_search == "Yes":
        # No relevant documents were found, so fall back to web search
        print(
            "---DECISION: RAG DOCS DO NOT CONTAIN RELEVANT CONTENT, FALLING BACK TO WEBSEARCH---"
        )
        web_fallback_status = "success"
        return "websearch"
    else:
        # We have relevant documents, so generate the answer
        print("---DECISION: GENERATE ANSWER---")
        return "generate"



In [52]:
def web_search(state):
    """
    Web search based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    
    # Check if 'documents' key exists in state, if not, initialize it
    if "documents" not in state:
        state["documents"] = []

    ## passes existing documents list to function, there may or may not be doc in the array
    
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})

    # Transform the keys from 'url' to 'source' and 'content' to 'page_content'
    transformed_docs = [{"source": d["url"], "page_content": d["content"]} for d in docs]

    # Create Document objects with the transformed results
    for doc in transformed_docs:
        document = Document(metadata={'source': doc['source']}, page_content=doc['page_content'])
        documents.append(document)


    # # Join the transformed documents into a single string
    # web_results = "\n".join([f"source: {d['source']}\npage_content: {d['page_content']}" for d in transformed_docs])

    # web_results = "\n".join([d["content"] for d in docs])
    # web_results = Document(page_content=web_results)

    ## adds the web results to the existing documents list
    
    # documents.append(web_results)

    # Status message
    global websearch_status
    websearch_status = "success"
    
    return {"documents": documents, "question": question}



In [53]:
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE AN ANSWER---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}




In [54]:
################################ Conditional Edge ##############################


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---HALLUCINATION CHECKER---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]


    ## SOURCE DOCUMENTS HANDLER ##
    ## create a new array with source file and page content snippet for use in GUI output
    
    global filtered_docs_formatted
    filtered_docs_formatted = []

    # for doc in documents:
    #     source = doc.metadata['source']  # Assuming 'metadata' is a dictionary with a 'source' key
    #     page_content_snippet = doc.page_content[:200]  # Get the first x number of characters of the snippet
    #     # filtered_docs_formatted.append(f"Source Document: {source}\n\nSnippet: {page_content_snippet}\n")

    #     ## this will be rendered in gradio as HTML with clickable source if URL
    #     filtered_docs_formatted.append(f'Source Document: <a href="{source}" target="_blank" class="custom-link">{source}</a>\n\n<br>Snippet: {page_content_snippet}<br><br>\n')

    base_url = 'http://172.16.6.3'

    thumbnail_path = 'images/thumbnails/folder-icon.jpg'
    
    for doc in documents:
        source = doc.metadata['source']  # Assuming 'metadata' is a dictionary with a 'source' key
        page_content_snippet = doc.page_content[:200]  # Get the first x number of characters of the snippet
        
        # Append the formatted HTML to the list
        filtered_docs_formatted.append(f'''
        <base href="{base_url}">

        <table>
            <tr>
                <td>
                <!-- <img src="{thumbnail_path}" alt="Thumbnail" style="width:100px;height:auto;"> -->
                
                <img src="{thumbnail_path}" alt="Thumbnail" style="width:auto;height:auto;">

                </td>
                <td>
                    <a href="{source}" target="_blank" class="custom-link">{source}</a><br>
                    Snippet: {page_content_snippet}<br><br>
                </td>
            </tr>
        </table>
        ''')


    ### execute hallucination grader function, 
    ### a grade of YES means grounded in documents.  
    ### a grade of NO would indicate not grounded in docs and would qualify as an hallucination.
    
    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]


    # Status message
    global hallucination_status, usefulness_status


    # Check whether or not it passed the hallucination check, yes is pass, no is fail.
    if grade == "yes":
        print("---DECISION: ANSWER IS GROUNDED IN DOCUMENTS - NO HALLUCINATIONS---")

        hallucination_status = "success"

        # Check question-answering
        print("---GRADE ANSWER vs THE QUESTION---")
        
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score["score"]

        ##  check whether or not the answer is related to the question
        
        if grade == "yes":
            print("---DECISION: ANSWER ADDRESSES QUESTION AND IS USEFUL---")
            usefulness_status = "success"
            return "useful"
        else:
            print("---DECISION: ANSWER DOES NOT ADDRESS QUESTION---")
            return "not useful"

    ## if it's hallucinating, and answer is not related to documents, retry
    else:
        print("---DECISION: POSSIBLE HALLUCINATIONS - ANSWER IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"



In [55]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes

workflow.add_node("websearch", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("rerank", rerank)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generate


# Langgraph Graph Build

In [56]:
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("retrieve", "rerank")
workflow.add_edge("rerank", "grade_documents")

# workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")

workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)

# Display graph of node and edge logic

In [57]:
# from IPython.display import Image, display
# from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

# app = workflow.compile()

# display(
#     Image(
#         app.get_graph().draw_mermaid_png(
#             draw_method=MermaidDrawMethod.API
#         )
#     )
# )

# Test Question Answer set

In [58]:
# %%time

# # Compile
# app = workflow.compile()

# # Test
# from pprint import pprint

# inputs = {"question": "Tell me about mental health and include any web search you need to add additional info."}
# inputs = {"question": "What methods could be used to treat covid in an older patient?"}
# inputs = {"question": "What year did the Bears football team win the super bowl?"}

# for output in app.stream(inputs):
#     for key, value in output.items():
#         pprint(f"Finished running: {key}:")
# pprint(value["generation"])

# print('\n' + 'Time to complete:')


# Agentic Response Function for GUI
Adding a bit of formatting as well for the status panel

In [59]:
import io
import sys
import time
from contextlib import redirect_stdout



### used to format and color the status alert

def status_update(status):
    if status == "success":
        return '<div style="background-color: green; color: white; padding: 5px; border-radius: 5px;">Completed successfully</div>'
    elif status in ["websearch", "vectorstore"]:
        return f'<div style="background-color: blue; color: white; padding: 5px; border-radius: 5px;">{status}</div>'
    else:
        return '<div style="background-color: grey; color: white; padding: 5px; border-radius: 5px;">Not used</div>'


In [60]:

### MAIN AGENTIC AGGREGATION FUNCTION

def get_agentic_response(question, basic_rag_toggle):

    ### reset value of previous variable values upon new execution of main function
    
    global router_status, router_choice, retrieve_status, rerank_status, relevance_status, web_fallback_status, websearch_status, hallucination_status, usefulness_status
    
    router_status = None
    router_choice = None
    retrieve_status = None
    rerank_status = None
    relevance_status = None
    web_fallback_status = None
    websearch_status = None
    hallucination_status = None
    usefulness_status = None

    # set the globals so that the previous calls to variables can use selected value in this function
    global filtered_docs_formatted


    model_id = "meta/llama-3.1-8b-instruct"
    
    # Compile the workflow
    app = workflow.compile()

    # Prepare the input
    inputs = {"question": question}

    # Initialize the response variable
    response = None

    # Create a string buffer to capture print statements
    buffer = io.StringIO()

    # Record the start time
    start_time = time.time()

    # Stream the output from the app
    with redirect_stdout(buffer):
        for output in app.stream(inputs):
            for key, value in output.items():
                # Check if 'generation' key is in the value
                if 'generation' in value:
                    response = value['generation']


    # Record the end time
    end_time = time.time()

    # Calculate the total processing time
    total_time = end_time - start_time

    # Get the captured print statements
    captured_output = buffer.getvalue()

    # Combine the response with the captured output and total processing time
    agent_output = f"Agents Steps:\n{captured_output}"
    graded_response = f"Graded Response:\n{response}"
    processing_time = f"Total Processing Time: {total_time:.2f} seconds"

    # Status messages for indicator panel

    router_status_result = status_update(router_status)
    router_choice_result = status_update(router_choice)
    retrieve_status_result = status_update(retrieve_status)
    rerank_status_result = status_update(rerank_status)
    relevance_status_result = status_update(relevance_status)
    web_fallback_status_result = status_update(web_fallback_status)
    websearch_status_result = status_update(websearch_status)
    hallucination_status_result = status_update(hallucination_status)
    usefulness_status_result = status_update(usefulness_status)
    vectorstore_files = get_unique_files(merged_documents)
    
    # if using rag-toggle
    if basic_rag_toggle:
        basic_rag_response, basic_rag_formatted_output_source_docs = get_rag_response(question)
        # agent_output = "Agent not used"
        relevance_status = "Agent not used"
        web_fallback_status = "Agent not used"
        websearch_status = "Agent not used"
        hallucination_status = "Agent not used"
        usefulness_status = "Agent not used"

        relevance_status_result = status_update(relevance_status)
        web_fallback_status_result = status_update(web_fallback_status)
        websearch_status_result = status_update(websearch_status)
        hallucination_status_result = status_update(hallucination_status)
        usefulness_status_result = status_update(usefulness_status)        
        rag_only_docs_content = "\n\n".join(basic_rag_formatted_output_source_docs)

        return basic_rag_response, processing_time, model_id, router_status_result, router_choice_result, retrieve_status_result, rerank_status_result, relevance_status_result, web_fallback_status_result, websearch_status_result, hallucination_status_result, usefulness_status_result, rag_only_docs_content, vectorstore_files


    ### bring in filtered docs formatted from outside function, join the contents to make it look better in textbox in GUI
    
    filtered_docs_content = "\n\n".join(filtered_docs_formatted)
    
    # not using rag toggle, then return all items in response
    
    return graded_response, processing_time, model_id, router_status_result, router_choice_result, retrieve_status_result, rerank_status_result, relevance_status_result, web_fallback_status_result, websearch_status_result, hallucination_status_result, usefulness_status_result, filtered_docs_content, vectorstore_files


### Example question array for GUI Example questions

In [61]:

# Edit data below for specific demos ----
EXAMPLE_TITLES = [
                     "### Vector Search",
                     "### Web Fallback",
                     "### Web Search",
                 ]
EXAMPLES = [

###Vector Search

               [
        "Create an email based on a summary of patient 20 and their experience at the clinic.  Please include all the details.",
        "What can you say about sunscreen effectiveness in preventing melanoma?",
        "Please summarize the clinical trial info we have on our drug ipilimumab for melanoma.",
        "What are the key domains of population-based approaches to mental health?",

               ],

### Web Fallback

               [
        "I am young person with COVID, what is my survival rate?",

               ],

### Web Search

		       [ 
        "What year did the Bears football team win the super bowl?",
        "What is the chemical makeup of water?"
               ],

        ]


# GUI setup

In [62]:
import gradio as gr

def clear_fields():
    return "", "", "", "", "", "", "", "", "","", "", "", "", "", ""


with gr.Blocks(theme=gr.themes.Soft(), title="Health Clinic Assistant") as demo:

    gr.HTML('''
    <style>
        .custom-html {
            border: 3px solid grey;
            border-radius: 10px; /* Adjust the value to change the roundness */
            padding: 10px; /* Optional: Adds some padding inside the border */
            height: 200px; /* Set a fixed height */
            overflow-y: auto; /* Optional: Adds a scrollbar if content overflows */            
        }

        .logo-container {
            display: flex;
            align-items: center;
        }
        .logo-container img {
            margin-right: 15px; /* Space between the image and the text */
        }     
        
        body, .gradio-container {
            background-color: black;
            color: white; /* Optional: Change text color to white for better contrast */
        }

        .custom-link {
            color: #DDA0DD; /* Light purple color */
            text-decoration: none;
        }

        .textbox_id textarea {
             color: red
        }
    </style>
    ''')

    
## TITLE
    with gr.Row():
        gr.HTML(f"""<div class="logo-container"><img src="/file=images/medical-symbol2.jpg" width="100" height="100" 
        style="float: left; 
        vertical-align: middle; 
        margin-right: 15px;">
        <h2>Agentic RAG Health Clinic Assistant</h2>
        </div>
        """)            


### Question input and RAG toggle panel
    
    with gr.Row():
        with gr.Column(scale=2):
            question = gr.Textbox(value="Enter your question", label="Question", lines=2, max_lines=2)
            with gr.Row():
                submit_button = gr.Button("Submit")
                clear_button = gr.Button("Clear")  # Add the Clear button next to Submit button     

        with gr.Column(scale=2):
            # llm_choice = gr.Dropdown(choices=list(llm_options.keys()), label="Select LLM", value="Nvidia-NIM-Llama-3.1-8b-instruct") # set default choice
            basic_rag_toggle = gr.Checkbox(label="Use Basic RAG")
            model_id = gr.Textbox(label="Model ID")
            processing_time = gr.Textbox(label="Processing Time")

    
# Add space and a section divider
    gr.Markdown("<hr>")
    
### Response section
    
    with gr.Row():
        with gr.Column(scale=2):
            response = gr.Textbox(label="Response", lines=10, max_lines=10)
        # with gr.Column():
        #     agent_output = gr.Textbox(label="Agent Steps", lines=10, max_lines=10)
        with gr.Column(scale=1):
            gr.Markdown("#### Router Status")  # Add this line for the title
            router_status_result = gr.HTML(label="Router Status")
            gr.Markdown("#### Router Choice")  # Add this line for the title
            router_choice_result = gr.HTML(label="Router Choice")
            gr.Markdown("#### Retrieve Status")  # Add this line for the title
            retrieve_status_result = gr.HTML(label="Retrieve Status")
            gr.Markdown("#### Rerank Status")  # Add this line for the title
            rerank_status_result = gr.HTML(label="Rerank Status")
        with gr.Column(scale=1):
            gr.Markdown("#### Relevance Check")  # Add this line for the title
            relevance_status_result = gr.HTML(label="Relevance Check")
            gr.Markdown("#### Web Fallback Check")  # Add this line for the title
            web_fallback_status_result = gr.HTML(label="Web Fallback Check")
            gr.Markdown("#### Web Search")  # Add this line for the title
            websearch_status_result = gr.HTML(label="Web Search")
            gr.Markdown("#### Hallucination Check")  # Add this line for the title
            hallucination_status_result = gr.HTML(label="Hallucination Check")
            gr.Markdown("#### Usefulness Check")  # Add this line for the title
            usefulness_status_result = gr.HTML(label="Usefulness Check")    
    
# Add space and a section divider
    gr.Markdown("<hr>")

### source documents HTML panel

    
    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("#### Source Documents")  # Add this line for the title
            source_documents = gr.HTML(label="Source Documents", elem_classes="custom-html")
        with gr.Column(scale=2):
            vectorstore_files = gr.Textbox(label="Uploaded Files", lines=10, max_lines=10)
            
    # with gr.Row():
    #     with gr.Column():
    #         vectorstore_files = gr.Textbox(label="Vectorstore Files", lines=10, max_lines=10)

    
    
    
# Add space and a section divider
    gr.Markdown("<hr>")

########## EXAMPLE QUICK PROMPT BUTTONS ###########	    
    def example_click(user_input):
        return user_input

    title_counter = 0
    for list_entry in EXAMPLES:
        if len(EXAMPLE_TITLES[title_counter]) > 0:
            gr.Markdown(EXAMPLE_TITLES[title_counter])
        with gr.Row():
            for entry in list_entry:
                button = gr.Button(entry)
                button.click(fn=example_click, inputs=button, outputs=question)
            title_counter += 1
####################################	


###  Gradio inputs go from the GUI --> to the function. 
###  Outputs are coming OUT of the function into the GUI in that order.  Naming doesn't matter.
###  the return statement of the function needs to return in the order that matches the outputs in gradio

    submit_button.click(
        get_agentic_response, 
        inputs=[question, basic_rag_toggle], 
        outputs=[response, processing_time, model_id, router_status_result, router_choice_result, retrieve_status_result, rerank_status_result, relevance_status_result, web_fallback_status_result, websearch_status_result, hallucination_status_result, usefulness_status_result, source_documents, vectorstore_files]
    )

    clear_button.click(
        clear_fields, 
        inputs=[], 
        outputs=[question, response, processing_time, model_id, router_status_result, router_choice_result, retrieve_status_result, rerank_status_result, relevance_status_result, web_fallback_status_result, websearch_status_result, hallucination_status_result, usefulness_status_result, source_documents, vectorstore_files]
    )    



demo.queue(max_size=5)
demo.launch(share=False, debug=True, server_name="172.16.6.3", server_port=7869, allowed_paths=["images/"])

Running on local URL:  http://172.16.6.3:7869

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.27.0, however version 4.44.1 is available, please upgrade.
--------

List of unique files in merged loader, sorted by file type:

http://172.16.6.3/docs/csv-powerscale-mount/healthcare.csv
http://172.16.6.3/docs/pdf-powerscale-mount/mental_health_first_aid.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_screening.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/investigating-the-efficacy-of-osimertinib-and-crizotinib-in-phase-3-clinical-trials-on-anti-cancer.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/covid_variants.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/understanding-pharmacology-covid-mrna.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_cells.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/ipilimumab-for-advanced-melanoma-a-pharmacologic-perspective.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/population_based_approach_to_mental_health.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_preventio

Traceback (most recent call last):
  File "/home/demo/miniconda3/envs/rag/lib/python3.10/site-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/home/demo/miniconda3/envs/rag/lib/python3.10/site-packages/gradio/route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/demo/miniconda3/envs/rag/lib/python3.10/site-packages/gradio/blocks.py", line 1788, in process_api
    result = await self.call_function(
  File "/home/demo/miniconda3/envs/rag/lib/python3.10/site-packages/gradio/blocks.py", line 1340, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/demo/miniconda3/envs/rag/lib/python3.10/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/demo/miniconda3/envs/rag/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    


List of unique files in merged loader, sorted by file type:

http://172.16.6.3/docs/csv-powerscale-mount/healthcare.csv
http://172.16.6.3/docs/pdf-powerscale-mount/mental_health_first_aid.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_screening.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/investigating-the-efficacy-of-osimertinib-and-crizotinib-in-phase-3-clinical-trials-on-anti-cancer.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/covid_variants.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/understanding-pharmacology-covid-mrna.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_cells.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/ipilimumab-for-advanced-melanoma-a-pharmacologic-perspective.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/population_based_approach_to_mental_health.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/skin_cancer_prevention_update.pdf
http://172.16.6.3/docs/pdf-powerscale-mount/covid_update.pdf
http://172.16.6.3/docs/pdf-powerscal